In [4]:
from utils import authenticate, get_venues, get_details, find_no_cancellation_fee, snipe
import pandas as pd
import re

from utils import book_reservation, cancel_reservation
import datetime

In [11]:
day = datetime.datetime(2023,2,11,17,30)
output = get_venues(day=day,limit=1)


{'lat': '40.7113', 'long': '-74.0077', 'day': '2023-02-11', 'party_size': 2, 'venue_id': None, 'limit': 1, 'bookmark': None}


In [12]:
data = output.get('data')  # Exclude HTTP Response Code
# Split each part of the response into smaller pieces to be dealt with separately, too much data in the response
query = data.get('query')
bookmark = data.get('bookmark')
results = data.get('results')


In [13]:
results_meta = results.get('meta')
venues = results.get('venues')


In [14]:
out = []
for venue in venues:

    venue_info = venue.get('venue')
    venue_name = venue_info.get('name')
    venue_location = venue_info.get('location')
    venue_id = venue_info.get('id').get('resy')

    slots = venue.get('slots')
    if slots:
        slots_df = pd.DataFrame(slots)
        slots_df_1 = pd.concat([slots_df, slots_df['config'].apply(
            lambda x: pd.Series(x).add_prefix('config_'))], axis=1)
        slots_df_2 = pd.concat([slots_df_1, slots_df_1['date'].apply(
            lambda x: pd.Series(x).add_prefix('date_'))], axis=1)
        slots_df_3 = pd.concat([slots_df_2, slots_df_2['payment'].apply(
            lambda x: pd.Series(x).add_prefix('payment_'))], axis=1)
        slots_df_3['resyId'] = venue_id
        slots_df_4 = pd.concat(
            [slots_df_3,slots_df_3['size'].apply(lambda x: pd.Series(x).add_prefix('size_'))], axis = 1
        )
        
    
    else:
        slots_df_4=pd.DataFrame()
    # slots_df_4 = pd.concat([slots_df_3,slots_df_3['config_token'].apply(lambda x: pd.Series(re.findall(r"(?<=resy/)\d+",x),index=['resyId']))],axis=1)
    out.append({'name': venue_name,
           'id': venue_id,
           'location': venue_location,
           'slots': slots_df_4}
              )


In [15]:
len(out)

1

In [16]:
out

[{'name': 'The Bar Room at The Beekman',
  'id': 40703,
  'location': {'time_zone': 'EST5EDT',
   'neighborhood': 'Financial District',
   'geo': {'lat': 40.711261, 'lon': -74.006851},
   'code': 'ny',
   'name': 'New York'},
  'slots':    availability                                             config  \
  0     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  1     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  2     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  3     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  4     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  5     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  6     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  7     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  8     {'id': 3}  {'id': 656585, 'type': 'The Bar Room', 'token'...   
  9     {'id': 3}  {'id': 656585, 'type': 'Th

In [10]:
for place in out:
    if place.get('name') == 'Dinner Party':
        print(place)

{'name': 'Dinner Party', 'id': 60228, 'location': {'time_zone': 'EST5EDT', 'neighborhood': 'Fort Greene', 'geo': {'lat': 40.6871027663847, 'lon': -73.9747080020124}, 'code': 'ny', 'name': 'New York'}, 'slots':    availability                                             config  \
0     {'id': 3}  {'id': 1053470, 'type': 'Communal', 'token': '...   
1     {'id': 3}  {'id': 1052063, 'type': 'Private', 'token': 'r...   
2     {'id': 3}  {'id': 1053470, 'type': 'Communal', 'token': '...   
3     {'id': 3}  {'id': 1052063, 'type': 'Private', 'token': 'r...   
4     {'id': 3}  {'id': 1053470, 'type': 'Communal', 'token': '...   
5     {'id': 3}  {'id': 1052063, 'type': 'Private', 'token': 'r...   
6     {'id': 3}  {'id': 1053470, 'type': 'Communal', 'token': '...   
7     {'id': 3}  {'id': 1052063, 'type': 'Private', 'token': 'r...   
8     {'id': 3}  {'id': 1053470, 'type': 'Communal', 'token': '...   
9     {'id': 3}  {'id': 1052063, 'type': 'Private', 'token': 'r...   
10    {'id': 3}  {'i

In [49]:
resy.keys()

Index(['availability', 'config', 'date', 'exclusive',
       'is_global_dining_access', 'floorplan', 'market', 'meta', 'pacing',
       'score', 'shift', 'size', 'status', 'table', 'template', 'time',
       'quantity', 'display_config', 'reservation_config', 'payment',
       'config_id', 'config_type', 'config_token', 'date_end', 'date_start',
       'payment_is_paid', 'payment_service_charge_options', 'resyId',
       'size_max', 'size_min'],
      dtype='object')

In [13]:
import datetime

date = datetime.date(2023,2,1)
time = datetime.time(7, 30)
combo = datetime.datetime.combine(date, time)
slots = tests[0]["slots"]
party_size = 2
slots["date_start"] = pd.to_datetime(slots["date_start"])
print(combo + datetime.timedelta(minutes=-30))
print(combo + datetime.timedelta(minutes=30))
slots[
    (slots["size_min"] <= party_size)
    & (slots["size_max"] >= party_size)
    & (
        (slots["date_start"]
        >= combo + datetime.timedelta(minutes=-30))
        & (slots["date_start"] <= combo + datetime.timedelta(minutes=30))
    )
]



NameError: name 'tests' is not defined

In [15]:
datetime.datetime(2023,1,20,7,30)

datetime.datetime(2023, 1, 20, 7, 30)

In [14]:
snipe(venue_id=60228,dateTime=datetime.datetime(2023,1,20,7,30),party_size=2)

{'lat': '40.7113', 'long': '-74.0077', 'day': <built-in method date of datetime.datetime object at 0x0000024924103B40>, 'party_size': 2, 'venue_id': 60228, 'limit': None, 'bookmark': None}


AttributeError: 'str' object has no attribute 'get'